In [ ]:
import numpy as np
import sympy as sym
import time
import threading
from copy import deepcopy

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation as ani
from matplotlib import cm, colors, rc
import matplotlib.ticker as ticker

In [ ]:
rc("animation", html = "jshtml")
plt.rcParams['figure.dpi'] = 360
sym.init_printing()
matplotlib.rcParams['animation.embed_limit'] = 2**128
G = 1

### Simple n-body simulation

$$F_i = G\frac{m_i m_j}{|r|^2} \hat{r}$$
$$\hat{r} = \frac{r}{|r|}$$
$$F_i = G\frac{m_i m_j}{|r|^2} \frac{r}{|r|} = G\frac{m_i m_j}{|r|^3} r$$
$$G=1$$
$$F_i = \sum_{j=0, j \neq i}^{N-1} \frac{m_i m_j}{|r|^3} r $$
$$a =  \frac{1}{m_i}  \sum_{j=0, j \neq i}^{N-1} \frac{m_i m_j}{|r|^3} r $$
$$a = \sum_{j=0, j \neq i}^{N-1} \frac{m_j}{|r|^3} r $$

Numerical approximation, there's no analytical solution:

$$a(t) = \frac{dv}{dt} \approx \frac{v(t + \Delta t) - v(t)}{\Delta t}$$
$$a_n \approx \frac{v_{n+1}-v_n}{\Delta t}$$

Euler method*
$$v_{n+1} \approx v_n + \Delta t \cdot a_n$$
$$p_{n+1} \approx p_n + \Delta t \cdot v_n$$

$$O(n^2) \rightarrow O(nlog(n))$$

In [ ]:
class Body:
    def __init__(self, pos, vel, mass):
        self.pos = pos
        self.vel = vel
        self.acc = np.array([0.0, 0.0])
        self.mass = mass

    def update(self, dt):
        self.pos += self.vel * dt
        self.vel += self.acc * dt
        self.acc = np.array([0.0, 0.0])

In [ ]:
def rand_disc():
    theta = random.uniform(0, 2 * np.pi)
    return np.array([np.cos(theta), np.sin(theta)]) * random.random()

def rand_body():
    pos = rand_disc()
    vel = rand_disc()
    return Body(pos, vel, 1.0)


In [ ]:
class Simulation:
    def __init__(self, seed):
        random.seed(seed)
        self.bodies = [rand_body() for _ in range(3)]

        vel = sum((b.vel * b.mass for b in self.bodies), np.array([0.0, 0.0])) / 3
        pos = sum((b.pos * b.mass for b in self.bodies), np.array([0.0, 0.0])) / 3

        for b in self.bodies:
            b.vel -= vel
            b.pos -= pos

        r = max(np.linalg.norm(b.pos) for b in self.bodies)
        for b in self.bodies:
            b.pos /= r

    def update(self):
        for i, b1 in enumerate(self.bodies):
            for b2 in self.bodies[i+1:]:
                r = b2.pos - b1.pos
                mag_sq = np.dot(r, r)
                mag = np.sqrt(mag_sq)
                min_dist = max(mag_sq, 0.0001)
                force_dir = r / (min_dist * mag)

                b1.acc += b2.mass * force_dir
                b2.acc -= b1.mass * force_dir

        for b in self.bodies:
            b.update(0.0001)

In [ ]:
class Body:
    def __init__(self, pos, vel, mass):
        self.pos = np.array(pos, dtype=np.float64)
        self.vel = np.array(vel, dtype=np.float64)
        self.mass = mass
        self.trail = [self.pos.copy()]

    def update(self, bodies, dt):
        acc = np.array([0.0, 0.0], dtype=np.float64)
        for b in bodies:
            if b is not self:
                delta = b.pos - self.pos
                r2 = np.dot(delta, delta)
                acc += G * b.mass * delta / r2**1.5
        self.vel += acc * dt
        self.pos += self.vel * dt
        self.trail.append(self.pos.copy())

def init_plot():
    ax.axis('off')
    ax.set_facecolor('black')
    for trail_line in trail_lines:
        trail_line.set_data([], [])
    return lines + trail_lines

def update(frame):
    min_x, max_x = float('inf'), float('-inf')
    min_y, max_y = float('inf'), float('-inf')

    for i, b in enumerate(bodies):
        b.update(bodies, 0.01)
        lines[i].set_data(b.pos[0], b.pos[1])

        x_trail, y_trail = zip(*b.trail)
        trail_lines[i].set_data(x_trail, y_trail)

        min_x = min(min_x, min(x_trail), b.pos[0])
        max_x = max(max_x, max(x_trail), b.pos[0])
        min_y = min(min_y, min(y_trail), b.pos[1])
        max_y = max(max_y, max(y_trail), b.pos[1])

    margin = 0.1 * max(max_x - min_x, max_y - min_y)
    ax.set_xlim(min_x - margin, max_x + margin)
    ax.set_ylim(min_y - margin, max_y + margin)

    return lines + trail_lines

bodies = [Body(np.random.rand(2)*2-1, np.random.rand(2)*0.1-0.05, 1) for _ in range(20)]

fig, ax = plt.subplots(facecolor='black')
lines = [ax.plot([], [], 'o', markersize=5, markerfacecolor='white', markeredgecolor='blue', alpha=0.9)[0] for _ in bodies]
trail_lines = [ax.plot([], [], linestyle='--', linewidth=0.5, color='white')[0] for _ in bodies]

FuncAnimation(fig, update, frames=np.arange(1000), init_func=init_plot, blit=True)

### Runge-Kutta 4

\begin{equation}
\ddot x_1 = \frac{-k x_1 + m_2 \dot \theta^2 l \sin \theta}{ m_1+m_2}- \frac{m_2 (k x_1 - m_2 \dot \theta^2 l \sin \theta -g(m_1 + m_2) \tan \theta )}{ (m_1+m_2) (m_1\tan^2 \theta + m_2) }
\end{equation}

\begin{equation}
\ddot \theta = \frac{k x_1 - m_2 \dot \theta^2 l \sin \theta - g(m_1+m_2) \tan \theta}{ l(m_1+m_2)\sin \theta \tan \theta + m_1 l \cos \theta }
\end{equation}

\\
$$l_0 = 1 \text{ m}, \quad l = 1 \text{ m}, \quad m_1 = 8 \text{ kg}, \quad m_2 = 10 \text{ kg}$$
$$\quad k = 300 \text{ N/m}, \quad \theta_0= 70^\circ, \quad \omega_0 = 0 \quad T_\text{total} = 5 \text{ s}, \quad x_{10} = 1 \text{ m }$$
$$dt = 0.005 \text{ s }, v_0 = 0$$

In [ ]:
g, l, m1, m2, k, l0 = 9.8, 1, 8, 10, 300, 1
t = 5
puntos = int(t*50)
theta = 70 * np.pi/180
x = 1
x2, y2 = x + l * np.sin(theta), - l * np.cos(theta)
v, omega = 0.0, 0.0

In [ ]:
start_time = time.time()

def runge_kutta_4th_order(h, t0, tf, y0):

    num_steps = int((tf - t0) / h)
    t_values = np.linspace(t0, tf, num_steps + 1)
    y_values = np.zeros((num_steps + 1, 4))
    y_values[0] = y0

    def f(t, y):
        x1, x1_dot, theta, theta_dot = y

        x1_ddot = (-k*x1 + m2*theta_dot**2*l*np.sin(theta))/(m1+m2) - (m2*(k*x1 - m2*theta_dot**2*l*np.sin(theta) - g*(m1+m2)*np.tan(theta)))/((m1+m2)*(m1*np.tan(theta)**2 + m2))
        theta_ddot = (k*x1 - m2*theta_dot**2*l*np.sin(theta) - g*(m1+m2)*np.tan(theta))/(l*(m1+m2)*np.sin(theta)*np.tan(theta) + m1*l*np.cos(theta))

        return np.array([x1_dot, x1_ddot, theta_dot, theta_ddot])

    for i in range(num_steps):
        t = t_values[i]
        y = y_values[i]

        k1 = h * f(t, y)
        k2 = h * f(t + h/2, y + k1/2)
        k3 = h * f(t + h/2, y + k2/2)
        k4 = h * f(t + h, y + k3)

        y_values[i+1] = y + (k1 + 2*k2 + 2*k3 + k4)/6

    return t_values, y_values

y0 = np.array([x, v, theta, omega])

t_values, y_values = runge_kutta_4th_order(t/puntos, 0, t, y0)

x1_values = y_values[:, 0]
theta_values = y_values[:, 2]

x_pendulo = x1_values + l * np.sin(theta_values)
y_pendulo = -l * np.cos(theta_values)

In [ ]:
fig, ax = plt.subplots(figsize=(7.5, 2.5))
ax.set_xlim(-l0-l, l0+l+1)
ax.set_ylim(-l, l)

line_resorte, = ax.plot([], [], 'r-o', markersize=6, linewidth=3)
line_pendulo, = ax.plot([], [], 'r-o', markersize=6, linewidth=3)
line_trayectoria, = ax.plot([], [], 'b-')

def init():
    line_resorte.set_data([], [])
    line_pendulo.set_data([], [])
    line_trayectoria.set_data([], [])
    return line_resorte, line_pendulo, line_trayectoria

def update(frame):
    x_resorte = [0, x1_values[frame]+1]
    y_resorte = [0, 0]
    x_pendulo = [x1_values[frame]+1, x1_values[frame]+1 + l * np.sin(theta_values[frame])]
    y_pendulo = [0, -l * np.cos(theta_values[frame])]

    line_resorte.set_data(x_resorte, y_resorte)
    line_trayectoria.set_data(x1_values[:frame+1]+1 + l * np.sin(theta_values[:frame+1]), -l * np.cos(theta_values[:frame+1]))
    line_pendulo.set_data(x_pendulo, y_pendulo)
    ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.25))

    return line_resorte, line_pendulo, line_trayectoria

ani.FuncAnimation(fig, update, frames=len(t_values), init_func=init, blit=True)

### 1D Wave function

In [ ]:
dx, L, t_max, c = 0.02, 1.0, 1.5, 1.8
l0, lL = 0, 0

def pos_ini(x):
  sig = 0.1
  return 1.0/np.sqrt(sig)*np.exp(-((x-0.3)**2)/sig**2)

def vel_ini(x):
  return np.zeros(len(x))

In [ ]:
def wave_1d(dx,t_max,L,c,l0,lL):
    dt = dx / (1.5*c)
    r = (c*dt/dx)**2
    a = 2*(1-r)

    x = np.linspace(0, L, num = 1+int(np.round((L)/dx)))
    lt, lx = 1+int(np.round((t_max)/dt)), len(x)
    U = np.zeros([lt,lx])

    U[0,:] = pos_ini(x)
    v0 = vel_ini(x)
    U[:,0], U[:, -1] = l0, lL
    for i in range(1,lx-1):
        U[1,i] = 0.5*a*U[0,i] + 0.5*r*(U[0,i-1] + U[0,i+1]) + dt*v0[i]

    for n in range(1,lt-1):
        for i in range(1,lx-1):
            U[n+1,i] = a*U[n,i] + r*(U[n,i-1] + U[n,i+1]) - U[n-1,i]

    return x, U

In [ ]:
x, U = wave_1d(dx,t_max,L,c,l0,lL)

def update(num_frame, U, line):
  line.set_ydata(U[num_frame,:])
  return line,

def create_animation(x, U):
  N = U.shape[0]
  fig = plt.figure()
  ax = plt.gca()
  ax.set_xlim([0,L])
  ax.set_ylim([np.min(U)-1,np.max(U)+1])

  line, = ax.plot(x,U[0],'r')
  plt.tight_layout()
  ani = animation.FuncAnimation(fig, update, N, fargs=(U,line))
  return ani

create_animation(x, U)

### 2D Wave function

Bidimensional wave equation with fixed ends at zero for the initial position $p(x,y)$

$$p(x,y) = \frac{1}{\sqrt{\sigma}} e^{\left( -{\frac{(x-0.5)^{2} +(y-0.5)^{2}}{\sigma^{2}}} \right)}$$

With $\sigma = 0.1$, domain $x \in [0, 1]$,  $y \in [0, 1]$ and initial velocity of each node equal to zero for $t=0$

|  |                                        |
|-----------------------|----------------------------------------|
| $$c$$                 |                                $$1.8$$ |
| $$dx$$                |                               $$0.05$$ |
| $$dy$$                |                               $$0.05$$ |
| $$t_{total}$$           |                                  $$0.5$$ |
| $$dt$$                | $$\frac{1}{10c}\frac{dx^2dy^2}{dx^2+dy^2}$$ |

In [ ]:
sigma, c, L  = 0.1, 1.8, 1
dx, dy = 0.05, 0.05
lx, ly = 1.0, 1.0
t_total = 0.5
dt = (1/(10*c)) * (((dx**2)*(dy**2))/((dx**2)+(dy**2)))
pts = 50*dt
l0, lL, lT, lB = 0, 1, 1, 0

In [ ]:
def pos_ini(x,y):
    return (1.0 / np.sqrt(sigma)) * np.exp(-((x - 0.5)**2 + (y - 0.5)**2) / sigma**2)

def vel_ini(x,y):
    return np.zeros([len(x),len(y)])

def solucion_ecuacion_onda_2D(dx, dy, t_max, L, c, l0, lL):

    rx, ry = (c*dt/dx)**2, (c*dt/dy)**2
    a = 2*(1-rx-ry)

    x, y = np.linspace(0, L, num = 1+int(np.round((L)/dx))), np.linspace(0, L, num = 1+int(np.round((L)/dy)))
    x, y = np.meshgrid(x, y)

    lt = 1+int((t_max)/dt)
    lx, ly = len(x), len(y)

    U = np.zeros([lt,lx,ly])
    U[0] = pos_ini(x, y)
    U[:, 0, :] = l0
    U[:, -1, :] = lL
    U[:, :, 0] = l0
    U[:, :, -1] = lL

    v0 = vel_ini(x,y)
    U[1,:,:] = U[0,:,:] + dt*v0

    for n in range(1, lt-1):
        for i in range(1, lx-1):
            for j in range(1, ly-1):
                U[n+1,i,j] = a*U[n,i,j] - U[n-1,i,j] + rx*(U[n,i+1,j]+U[n,i-1,j]) + ry*(U[n,i,j+1]+U[n,i,j-1])

    return x, y, U

In [ ]:
start_time = time.time()
x, y, U = solucion_ecuacion_onda_2D(dx, dy, t_total, L, c, 0, 0)

print(f't: {time.time()-start_time} (s)')
print(f'x.shape:{x.shape}, y.shape:{y.shape} Ux.shape:{U.shape}')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

def update(frame):
    ax.clear()
    ax.set_title(f't = {round(frame*dt,2)} s')
    ax.set_zlim3d(np.min(U)-1, np.max(U)+1)

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    ax.plot_surface(x, y, U[frame], cmap='viridis')

    return ax

animation.FuncAnimation(fig, update, frames=range(0, int(t_total/dt), 50), interval=50)